# Chapter 4: Reflection (LangChain 代码示例)

本 notebook 演示了如何使用 LangChain 实现 Reflection（反思）模式，通过生成-批判-改进的链式流程来优化产品描述。

## 前置要求

- Python 3.8+
- 阿里云 DashScope API Key（需要设置环境变量 `DASHSCOPE_API_KEY`）


## 1. 安装依赖

首先安装所需的 Python 包。


In [1]:
# 安装必要的依赖包（使用 %pip 在 Jupyter 中更推荐）
%pip install -q langchain-core>=0.1.0 langchain-community>=0.1.0 dashscope>=1.17.0 python-dotenv>=1.0.0 nest-asyncio


Note: you may need to restart the kernel to use updated packages.


## 2. 获取 API Key

如果环境变量或 .env 文件中没有设置 `DASHSCOPE_API_KEY`，可以在此处直接设置。


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

# 获取 API key（优先从环境变量读取，如果没有则从 .env 文件）
dashscope_api_key = os.getenv("DASHSCOPE_API_KEY")
if not dashscope_api_key:
    print("警告：未找到 DASHSCOPE_API_KEY，请确保已设置环境变量或 .env 文件")


## 3. 导入库并初始化模型


In [3]:
import asyncio
import nest_asyncio
from langchain_community.chat_models import ChatTongyi
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 允许在 Jupyter notebook 中运行异步代码
nest_asyncio.apply()

# --- Configuration ---
# 初始化 Qwen 语言模型（使用阿里云 DashScope API）
# model 参数可以指定不同的 Qwen 模型，如 "qwen-flash", "qwen-turbo", "qwen-plus", "qwen-max" 等
try:
    llm = ChatTongyi(
        model="qwen-flash",
        temperature=0.7,
        dashscope_api_key=dashscope_api_key
    )
    print(f"Language model initialized: qwen-flash")
except Exception as e:
    print(f"Error initializing language model: {e}")
    print("Please ensure your DASHSCOPE_API_KEY is set correctly.")
    llm = None


# --- Define Chain Components ---

# 1. Initial Generation: Creates the first draft of the product description.
# The input to this chain will be a dictionary, so we update the prompt template.
generation_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Write a short, simple product description for a new smart coffee mug."),
        ("user", "{product_details}")
    ])
    | llm
    | StrOutputParser()
)

# 2. Critique: Evaluates the generated description and provides feedback.
critique_chain = (
    ChatPromptTemplate.from_messages([
        ("system", """Critique the following product description based on clarity, conciseness, and appeal.
        Provide specific suggestions for improvement."""),
        # This will receive 'initial_description' from the previous step.
        ("user", "Product Description to Critique:\n{initial_description}")
    ])
    | llm
    | StrOutputParser()
)

# 3. Refinement: Rewrites the description based on the original details and the critique.
refinement_chain = (
    ChatPromptTemplate.from_messages([
        ("system", """Based on the original product details and the following critique,
        rewrite the product description to be more effective.

        Original Product Details: {product_details}
        Critique: {critique}

        Refined Product Description:"""),
        ("user", "") # User input is empty as the context is provided in the system message
    ])
    | llm
    | StrOutputParser()
)


# --- Build the Full Reflection Chain (Refactored) ---
# This chain is structured to be more readable and linear.
full_reflection_chain = (
    RunnablePassthrough.assign(
        initial_description=generation_chain
    )
    | RunnablePassthrough.assign(
        critique=critique_chain
    )
    | refinement_chain
)


# --- Run the Chain ---
async def run_reflection_example(product_details: str):
    """Runs the LangChain reflection example with product details."""
    if not llm:
        print("LLM 未初始化，无法运行反思示例。请检查 API key 配置。")
        return
    
    print(f"\n--- Running Reflection Example for Product: '{product_details}' ---")
    try:
        # The chain now expects a dictionary as input from the start.
        final_refined_description = await full_reflection_chain.ainvoke(
            {"product_details": product_details}
        )
        print("\n--- Final Refined Product Description ---")
        print(final_refined_description)
    except Exception as e:
        print(f"\nAn error occurred during chain execution: {e}")

# 运行反思示例
# 在 Jupyter notebook 中，可以直接使用 await run_reflection_example(test_product_details)
# 或者使用 asyncio.run()（需要 nest_asyncio 支持）
if llm:
    test_product_details = "A mug that keeps coffee hot and can be controlled by a smartphone app."
    asyncio.run(run_reflection_example(test_product_details))
else:
    print("LLM 未初始化，无法运行示例。请检查 API key 配置。")

Language model initialized: qwen-flash

--- Running Reflection Example for Product: 'A mug that keeps coffee hot and can be controlled by a smartphone app.' ---

--- Final Refined Product Description ---
**SmartSip Mug – Your Coffee, Perfectly Hot, Every Time.**

No more cold coffee by 10 a.m. With double-wall vacuum insulation and intelligent heating, SmartSip keeps your brew steaming for up to 4 hours—starting from the moment you pour it in.

Control it with your phone: set your ideal temperature (120°F–160°F), schedule warming bursts, or get a gentle reminder when it’s just right. Whether you’re deep in a work session, on a morning commute, or hiking through the woods, your coffee stays at peak warmth—no guesswork, no disappointment.

Lightweight, sleek, and built for real life.  
Wake up to a perfect cup. Stay warm. Stay focused. Stay inspired.

---

### ✅ Why This Works:
- **Clear, specific benefits:** “From the moment you pour” removes ambiguity; “120°F–160°F” adds precision and 